In [ ]:
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import neuropacks as packs
import numpy as np
import os
import seaborn as sns

from noise_correlations import analysis, utils

%matplotlib inline

In [ ]:
base_path = '/Volumes/pss/fits/neurocorr'

In [ ]:
pvc11_1_path = os.path.join(base_path, 'exp4_1_tuned_values_pvc11_65_1000_1000.h5')
pvc11_2_path = os.path.join(base_path, 'exp4_2_tuned_values_pvc11_50_1000_1000.h5')
pvc11_3_path = os.path.join(base_path, 'exp4_3_tuned_values_pvc11_75_1000_1000.h5')
ac1_path = os.path.join(base_path, 'exp4_resp_values_ac1_50_1000_1000.h5')
ret2_path = os.path.join(base_path, 'exp4_tuned_values_ret2_50_1000_1000.h5')
optimality_path = os.path.join(base_path, 'neurocorr_optimality_results.h5')

In [ ]:
pvc11_1 = h5py.File(pvc11_1_path, 'r')
pvc11_2 = h5py.File(pvc11_2_path, 'r')
pvc11_3 = h5py.File(pvc11_3_path, 'r')
ret2 = h5py.File(ret2_path, 'r')
ac1 = h5py.File(ac1_path, 'r')
results = [pvc11_1, pvc11_2, pvc11_3, ret2, ac1]

In [ ]:
p_r_vals = [
    np.mean(
        result['v_lfi'][:][..., np.newaxis] > result['v_r_lfi'][:],
        axis=-1
    )
    for result in results
]

In [ ]:
[result.close() for result in results]

In [ ]:
with h5py.File(optimality_path, 'r') as f:
    pvc11_1_opt = f['pvc11_1'][:]
    pvc11_2_opt = f['pvc11_2'][:]
    pvc11_3_opt = f['pvc11_3'][:]
    ret2_opt = f['ret2'][:]
    ac1_opt = f['ac1'][:]

In [ ]:
optimals = [pvc11_1_opt, pvc11_2_opt, pvc11_3_opt, ret2_opt, ac1_opt]

In [ ]:
fig, axes = plt.subplots(3, 5, figsize=(18, 10), sharex=True, sharey=True)

for idx, ax in enumerate(axes[0]):
    ax.hexbin(
        x=optimals[idx][1],
        y=p_r_vals[idx][2],
        bins='log',
        gridsize=60,
        cmap='Greys'
    )
    
for idx, ax in enumerate(axes[1]):
    ax.hexbin(
        x=optimals[idx][8],
        y=p_r_vals[idx][9],
        gridsize=30,
        bins='log',
        cmap='Greys'
    )
    
for idx, ax in enumerate(axes[2]):
    ax.scatter(
        x=optimals[idx][18],
        y=p_r_vals[idx][19],
        color='gray'
    )
    
for ax in axes.ravel():
    ax.set_xlim([-1, 1])
    ax.set_ylim([0, 1])
    

axes[0, 0].set_title(r'\textbf{PVC11, 1}', fontsize=22)
axes[0, 1].set_title(r'\textbf{PVC11, 2}', fontsize=22)
axes[0, 2].set_title(r'\textbf{PVC11, 3}', fontsize=22)
axes[0, 3].set_title(r'\textbf{RET2}', fontsize=22)
axes[0, 4].set_title(r'\textbf{AC1}', fontsize=22)


fig.text(
    x=0.5, y=-0.3,
    s=r'\textbf{Mean-Variance Correlation for Optimal Orientation}',
    va='center',
    ha='center',
    transform=axes[2, 2].transAxes,
    fontsize=25)
fig.text(
    x=-0.5, y=0.5,
    s=r'\textbf{Percentile}',
    va='center',
    ha='center',
    transform=axes[1, 0].transAxes,
    rotation=90,
    fontsize=25)

axes[0, 0].set_ylabel(r'$D=3$', fontsize=25)
axes[1, 0].set_ylabel(r'$D=10$', fontsize=25)
axes[2, 0].set_ylabel(r'$D=20$', fontsize=25)
plt.savefig('optimality_plots.png', bbox_inches='tight')
plt.show()